## Introdução

Neste notebook, iremos detalhar a construção de um Grafo (Rede) que represente a malha aérea brasileira.

Ele será construído com auxílio de dados públicos disponibilizados pela ANAC sobre os voos que aconteceram em aeroportos brasileiros.

Partiremos dos dados estruturados (tabelas .csv) e, com auxílio das bibliotecas Pandas e NetworkX, os transformaremos em um grafo.

Esta base será utilizada em outro *Jupyter Notebook* onde focaremos em estudar os aspectos matemáticos e estatísticos da rede criada.

Este exercício tem intuito puramente acadêmico.

## Importando dados dos aeroportos

Dados com todos os aeroportos do mundo (tabela com as coordenadas dos aeroportos)

https://datahub.io/core/airport-codes#python


Dados com todos os vôos com origem ou destino no Brasil (dados públicos da [ANAC](https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/dados-estatisticos) )


In [1]:
!wget https://datahub.io/core/airport-codes/r/airport-codes.csv

--2021-08-06 19:12:43--  https://datahub.io/core/airport-codes/r/airport-codes.csv
Resolving datahub.io (datahub.io)... 104.21.40.221, 172.67.157.38, 2606:4700:3030::ac43:9d26, ...
Connecting to datahub.io (datahub.io)|104.21.40.221|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/airport-codes/airport-codes_csv/data/e07739e49300d125989ee543d5598c4b/airport-codes_csv.csv [following]
--2021-08-06 19:12:44--  https://pkgstore.datahub.io/core/airport-codes/airport-codes_csv/data/e07739e49300d125989ee543d5598c4b/airport-codes_csv.csv
Resolving pkgstore.datahub.io (pkgstore.datahub.io)... 104.21.40.221, 172.67.157.38, 2606:4700:3030::ac43:9d26, ...
Connecting to pkgstore.datahub.io (pkgstore.datahub.io)|104.21.40.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6232459 (5.9M) [text/csv]
Saving to: ‘airport-codes.csv’

airport-codes.csv   100%[===================>]   5.94M  11.0MB/s    in 0.5s    

20

In [2]:
!gdown https://drive.google.com/uc?id=13TdnsCxmsFxvDlWV2FJ_81GwKo4IVS0z

Downloading...
From: https://drive.google.com/uc?id=13TdnsCxmsFxvDlWV2FJ_81GwKo4IVS0z
To: /content/Dados Estatisticos.csv
108MB [00:00, 171MB/s] 


## Pré processando a base de dados

Nesta etapa, faremos todo o pré processamento necessário para a construção do nosso Grafo.

**Importando o conjunto de dados**

In [3]:
import pandas as pd
import numpy as np

dados_anac = pd.read_csv('https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/dados-estatisticos/arquivos/resumo_anual_2020.csv', sep=';', encoding='latin-1')

In [4]:
dados_anac.head()

,EMPRESA (SIGLA),EMPRESA (NOME),EMPRESA (NACIONALIDADE),ANO,MÊS,AEROPORTO DE ORIGEM (SIGLA),AEROPORTO DE ORIGEM (NOME),AEROPORTO DE ORIGEM (UF),AEROPORTO DE ORIGEM (REGIÃO),AEROPORTO DE ORIGEM (PAÍS),AEROPORTO DE ORIGEM (CONTINENTE),AEROPORTO DE DESTINO (SIGLA),AEROPORTO DE DESTINO (NOME),AEROPORTO DE DESTINO (UF),AEROPORTO DE DESTINO (REGIÃO),AEROPORTO DE DESTINO (PAÍS),AEROPORTO DE DESTINO (CONTINENTE),NATUREZA,GRUPO DE VOO,PASSAGEIROS PAGOS,PASSAGEIROS GRÁTIS,CARGA PAGA (KG),CARGA GRÁTIS (KG),CORREIO (KG),ASK,RPK,ATK,RTK,COMBUSTÍVEL (LITROS),DISTÂNCIA VOADA (KM),DECOLAGENS,CARGA PAGA KM,CARGA GRATIS KM,CORREIO KM,ASSENTOS,PAYLOAD,HORAS VOADAS,BAGAGEM (KG)
0,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2020,1,KDFW,"DALLAS & FORT WORTH, TEXAS",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGL,RIO DE JANEIRO,RJ,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,0.0,0.0,0.0,0.0,0.0,2559380.0,2071070.0,463045.0,258934.0,0.0,8419.0,1.0,4.875440e+07,0.0,23783700.0,304.0,55000.0,"10,783",0.0
1,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2020,1,KDFW,"DALLAS & FORT WORTH, TEXAS",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGR,GUARULHOS,SP,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,7379.0,401.0,267245.0,0.0,61649.0,72679100.0,58804500.0,13149200.0,7932760.0,0.0,239076.0,29.0,2.155430e+09,0.0,484945000.0,8816.0,1595000.0,"293,867",0.0
2,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2020,1,KJFK,"NEW YORK, NEW YORK",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGL,RIO DE JANEIRO,RJ,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,5978.0,365.0,57303.0,0.0,35.0,63068600.0,46204000.0,10434200.0,4601510.0,0.0,231870.0,30.0,4.428950e+08,0.0,270515.0,8160.0,1350000.0,"280,783",0.0
3,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2020,1,KJFK,"NEW YORK, NEW YORK",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGR,GUARULHOS,SP,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,7472.0,299.0,267201.0,0.0,5353.0,72225500.0,57265400.0,13067100.0,7242730.0,0.0,237584.0,31.0,2.047830e+09,0.0,41025400.0,9424.0,1705000.0,"285,766",0.0
4,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2020,1,KLAX,"LOS ANGELES, CALIFORNIA",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGR,GUARULHOS,SP,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,4076.0,99.0,86539.0,0.0,39018.0,45071900.0,40429800.0,7141680.0,4884080.0,0.0,158704.0,16.0,8.583800e+08,0.0,387020000.0,4544.0,720000.0,"191,583",0.0


In [5]:
print( f" Tamanho do dataset {len(dados_anac)}" )

 Tamanho do dataset 26735


**Visualizando as colunas e seus tipos**

In [6]:
dados_anac.dtypes

EMPRESA (SIGLA)                       object
EMPRESA (NOME)                        object
EMPRESA (NACIONALIDADE)               object
ANO                                    int64
MÊS                                    int64
AEROPORTO DE ORIGEM (SIGLA)           object
AEROPORTO DE ORIGEM (NOME)            object
AEROPORTO DE ORIGEM (UF)              object
AEROPORTO DE ORIGEM (REGIÃO)          object
AEROPORTO DE ORIGEM (PAÍS)            object
AEROPORTO DE ORIGEM (CONTINENTE)      object
AEROPORTO DE DESTINO (SIGLA)          object
AEROPORTO DE DESTINO (NOME)           object
AEROPORTO DE DESTINO (UF)             object
AEROPORTO DE DESTINO (REGIÃO)         object
AEROPORTO DE DESTINO (PAÍS)           object
AEROPORTO DE DESTINO (CONTINENTE)     object
NATUREZA                              object
GRUPO DE VOO                          object
PASSAGEIROS PAGOS                    float64
PASSAGEIROS GRÁTIS                   float64
CARGA PAGA (KG)                      float64
CARGA GRÁT

*As colunas do tipo 'object' referem-se a 'strings'*

**Selecionando os voos puramente brasileiros**

In [7]:
dados_anac_br = dados_anac.query( "`AEROPORTO DE ORIGEM (PAÍS)` == 'BRASIL' and `AEROPORTO DE DESTINO (PAÍS)` == 'BRASIL'" )

*Validando a seleção*

In [8]:
dados_anac_br[ ["AEROPORTO DE ORIGEM (PAÍS)", "AEROPORTO DE DESTINO (PAÍS)"] ].describe()

,AEROPORTO DE ORIGEM (PAÍS),AEROPORTO DE DESTINO (PAÍS)
count,18276,18276
unique,1,1
top,BRASIL,BRASIL
freq,18276,18276


## Agregando por par ORIGEM-DESTINO

Nesta etapa, construimos os pares únicos de aeroportos conectados por algum voo.

Os aeroportos serão os **nós** da nossa **rede**.
Dois aeroportos são conectados se existe ao menos um voo que os ligue.

Os pares finais vão conter:
  - Sigla, UF, Região e coordenadas de cada aeroporto
  - Total de voos entre eles
  - Total de voos feito por empresas brasileiras
  - Total de voos de natureza 'DOMÉSTICA'
  - Total de voos dos tipos *Improdutivo*, *Regulares* e *Não Regulares*  
  - Distância entre origem e destino

In [9]:
COLUMNS = ['AEROPORTO DE ORIGEM (SIGLA)',
           'AEROPORTO DE ORIGEM (UF)',
           'AEROPORTO DE ORIGEM (REGIÃO)',
           'AEROPORTO DE DESTINO (SIGLA)',
           'AEROPORTO DE DESTINO (UF)',
           'AEROPORTO DE DESTINO (REGIÃO)',
           'GRUPO DE VOO',
           'EMPRESA (NACIONALIDADE)',
           'DISTÂNCIA VOADA (KM)',
           'NATUREZA'
           ]

dados_anac_br = dados_anac_br[ COLUMNS ]

Criando variáveis auxiliares

In [10]:
dados_anac_br['VOO REGULAR'] = dados_anac_br['GRUPO DE VOO'] == 'REGULAR'
dados_anac_br['VOO NÃO REGULAR'] = dados_anac_br['GRUPO DE VOO'] == 'NÃO REGULAR'
dados_anac_br['IMPRODUTIVO'] = dados_anac_br['GRUPO DE VOO'] == 'IMPRODUTIVO'
dados_anac_br['EMPRESA NACIONAL'] = dados_anac_br['EMPRESA (NACIONALIDADE)'] == 'BRASILEIRA'
dados_anac_br['NATUREZA DOMÉSTICA'] =  dados_anac_br['NATUREZA'] == 'DOMÉSTICA'

dados_anac_br = dados_anac_br.drop( ['GRUPO DE VOO', 'EMPRESA (NACIONALIDADE)', 'NATUREZA'], axis=1 )

In [11]:
dados_gb = dados_anac_br.groupby( by=['AEROPORTO DE ORIGEM (SIGLA)', 'AEROPORTO DE DESTINO (SIGLA)'] )

In [12]:
dados_final = dados_gb[ ['AEROPORTO DE ORIGEM (UF)',
                         'AEROPORTO DE ORIGEM (REGIÃO)',
                         'AEROPORTO DE DESTINO (UF)',
                         'AEROPORTO DE DESTINO (REGIÃO)'] ].max()
dados_final[ 'DISTÂNCIA VOADA (KM)' ] = dados_gb[ 'DISTÂNCIA VOADA (KM)' ].max()
dados_final[ 'VOO REGULAR' ] = dados_gb[ 'VOO REGULAR' ].sum()
dados_final[ 'VOO NÃO REGULAR' ] = dados_gb[ 'VOO NÃO REGULAR' ].sum()
dados_final[ 'IMPRODUTIVO' ] = dados_gb[ 'IMPRODUTIVO' ].sum()
dados_final[ 'EMPRESA NACIONAL' ] = dados_gb[ 'EMPRESA NACIONAL' ].sum()
dados_final[ 'NATUREZA DOMÉSTICA' ] = dados_gb[ 'NATUREZA DOMÉSTICA' ].sum()
dados_final[ 'VOOS' ] = dados_final[ 'IMPRODUTIVO' ] + dados_final[ 'VOO REGULAR' ] + dados_final[ 'VOO NÃO REGULAR' ]

In [13]:
dados_final = dados_final.reset_index()

In [14]:
dados_final.head()

,AEROPORTO DE ORIGEM (SIGLA),AEROPORTO DE DESTINO (SIGLA),AEROPORTO DE ORIGEM (UF),AEROPORTO DE ORIGEM (REGIÃO),AEROPORTO DE DESTINO (UF),AEROPORTO DE DESTINO (REGIÃO),DISTÂNCIA VOADA (KM),VOO REGULAR,VOO NÃO REGULAR,IMPRODUTIVO,EMPRESA NACIONAL,NATUREZA DOMÉSTICA,VOOS
0,SBAC,SBFZ,CE,NORDESTE,CE,NORDESTE,720.0,2,0,0,2,2,2
1,SBAC,SBKP,CE,NORDESTE,SP,SUDESTE,11410.0,1,0,0,1,1,1
2,SBAE,SBAE,SP,SUDESTE,SP,SUDESTE,0.0,0,0,2,2,2,2
3,SBAE,SBBU,SP,SUDESTE,SP,SUDESTE,21.0,0,1,0,1,1,1
4,SBAE,SBGL,SP,SUDESTE,RJ,SUDESTE,602.0,1,0,0,1,1,1


**Adicionando informações de coordenadas**

In [16]:
dados_coord = pd.read_csv('airport-codes.csv')

In [17]:
dados_coord.columns

Index(['ident', 'type', 'name', 'elevation_ft', 'continent', 'iso_country',
       'iso_region', 'municipality', 'gps_code', 'iata_code', 'local_code',
       'coordinates'],
      dtype='object')

In [18]:
dados_coord = dados_coord[ ['gps_code', 'coordinates'] ]

dados_final = dados_final.merge( dados_coord, right_on = 'gps_code', left_on='AEROPORTO DE ORIGEM (SIGLA)' )
dados_final = dados_final.rename( columns={'coordinates':'COORDENADAS_ORIGEM'} )

dados_final = dados_final.merge( dados_coord, right_on = 'gps_code', left_on='AEROPORTO DE DESTINO (SIGLA)' )
dados_final = dados_final.rename( columns={'coordinates':'COORDENADAS_DESTINO'} )

In [19]:
dados_final

,AEROPORTO DE ORIGEM (SIGLA),AEROPORTO DE DESTINO (SIGLA),AEROPORTO DE ORIGEM (UF),AEROPORTO DE ORIGEM (REGIÃO),AEROPORTO DE DESTINO (UF),AEROPORTO DE DESTINO (REGIÃO),DISTÂNCIA VOADA (KM),VOO REGULAR,VOO NÃO REGULAR,IMPRODUTIVO,EMPRESA NACIONAL,NATUREZA DOMÉSTICA,VOOS,gps_code_x,COORDENADAS_ORIGEM,gps_code_y,COORDENADAS_DESTINO
0,SBAC,SBFZ,CE,NORDESTE,CE,NORDESTE,720.0,2,0,0,2,2,2,SBAC,"-37.804722, -4.568611",SBFZ,"-38.53260040283203, -3.776279926300049"
1,SBBE,SBFZ,PA,NORTE,CE,NORDESTE,78384.0,19,7,0,26,26,26,SBBE,"-48.4762992859, -1.3792500495900002",SBFZ,"-38.53260040283203, -3.776279926300049"
2,SBBR,SBFZ,DF,CENTRO-OESTE,CE,NORDESTE,203040.0,22,11,2,33,33,35,SBBR,"-47.920834, -15.869167",SBFZ,"-38.53260040283203, -3.776279926300049"
3,SBCF,SBFZ,MG,SUDESTE,CE,NORDESTE,59456.0,17,16,0,33,33,33,SBCF,"-43.97194290161133, -19.62444305419922",SBFZ,"-38.53260040283203, -3.776279926300049"
4,SBCT,SBFZ,PR,SUL,CE,NORDESTE,2673.0,0,2,0,2,2,2,SBCT,"-49.1758003235, -25.5284996033",SBFZ,"-38.53260040283203, -3.776279926300049"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,SBSP,SDIM,SP,SUDESTE,SP,SUDESTE,610.0,1,0,0,1,1,1,SBSP,"-46.65638732910156, -23.626110076904297",SDIM,"-46.78555679321289, -24.164722442626953"
1799,SBSP,SDUB,SP,SUDESTE,SP,SUDESTE,1944.0,1,0,0,1,1,1,SBSP,"-46.65638732910156, -23.626110076904297",SDUB,"-45.075599670410156, -23.441099166870117"
1800,SBSP,SJTC,SP,SUDESTE,SP,SUDESTE,8584.0,3,0,0,3,3,3,SBSP,"-46.65638732910156, -23.626110076904297",SJTC,"-55.926764, -22.249873"
1801,SBSP,SNBA,SP,SUDESTE,SP,SUDESTE,18078.0,3,0,0,3,3,3,SBSP,"-46.65638732910156, -23.626110076904297",SNBA,"-48.594100952148, -20.584499359131"


In [20]:
dados_final = dados_final.drop( ['gps_code_x', 'gps_code_y'], axis=1 )

In [21]:
dados_final['LAT_ORIGEM']  = dados_final['COORDENADAS_ORIGEM'].apply( lambda x: float(x.split(',')[0]) )
dados_final['LONG_ORIGEM']  = dados_final['COORDENADAS_ORIGEM'].apply( lambda x: float(x.split(',')[1]) )
dados_final = dados_final.drop( ['COORDENADAS_ORIGEM'], axis=1 )

dados_final['LAT_DESTINO']  = dados_final['COORDENADAS_DESTINO'].apply( lambda x: float(x.split(',')[0]) )
dados_final['LONG_DESTINO']  = dados_final['COORDENADAS_DESTINO'].apply( lambda x: float(x.split(',')[1]) )
dados_final = dados_final.drop( ['COORDENADAS_DESTINO'], axis=1 )

**Renomeando colunas para caracteres ASCII e removendo espaços**

https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string

In [22]:
import unicodedata

def fix_columns_name(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').replace(' ', '_').replace('(', '').replace(')', '')

In [23]:
dados_final = dados_final.rename( columns= { col:fix_columns_name(col) for col in dados_final.columns } )

**Removendo valores NaN**

In [24]:
print( f"Tamanho original - {len(dados_final)}" )
dados_final = dados_final.dropna()
print( f"Tamanho final - {len(dados_final)}" )

Tamanho original - 1803
Tamanho final - 1684


**Resultado Final**

In [25]:
dados_final.head()

,AEROPORTO_DE_ORIGEM_SIGLA,AEROPORTO_DE_DESTINO_SIGLA,AEROPORTO_DE_ORIGEM_UF,AEROPORTO_DE_ORIGEM_REGIAO,AEROPORTO_DE_DESTINO_UF,AEROPORTO_DE_DESTINO_REGIAO,DISTANCIA_VOADA_KM,VOO_REGULAR,VOO_NAO_REGULAR,IMPRODUTIVO,EMPRESA_NACIONAL,NATUREZA_DOMESTICA,VOOS,LAT_ORIGEM,LONG_ORIGEM,LAT_DESTINO,LONG_DESTINO
0,SBAC,SBFZ,CE,NORDESTE,CE,NORDESTE,720.0,2,0,0,2,2,2,-37.804722,-4.568611,-38.5326,-3.77628
1,SBBE,SBFZ,PA,NORTE,CE,NORDESTE,78384.0,19,7,0,26,26,26,-48.476299,-1.379250,-38.5326,-3.77628
2,SBBR,SBFZ,DF,CENTRO-OESTE,CE,NORDESTE,203040.0,22,11,2,33,33,35,-47.920834,-15.869167,-38.5326,-3.77628
3,SBCF,SBFZ,MG,SUDESTE,CE,NORDESTE,59456.0,17,16,0,33,33,33,-43.971943,-19.624443,-38.5326,-3.77628
4,SBCT,SBFZ,PR,SUL,CE,NORDESTE,2673.0,0,2,0,2,2,2,-49.175800,-25.528500,-38.5326,-3.77628


## Construindo o Grafo com o NetworkX

A partir desse ponto, já podemos consturir nosso grafo com o NetworkX.

**Adicionando as conexões entre os nós**

In [26]:
import networkx as nx
from pprint import pprint

In [27]:
graph_voos = nx.from_pandas_edgelist(dados_final,  
                                     source ='AEROPORTO_DE_ORIGEM_SIGLA',
                                     target ='AEROPORTO_DE_DESTINO_SIGLA',
                                     edge_attr = ['DISTANCIA_VOADA_KM', 'VOO_REGULAR', 'VOO_NAO_REGULAR',
                                                  'IMPRODUTIVO', 'EMPRESA_NACIONAL', 'NATUREZA_DOMESTICA', 'VOOS'],
                                     create_using = nx.DiGraph())

**Adicionando os atributos dos nós**

Formatando adequadamente o nome das colunas

In [28]:
cols = dados_final.columns
dados_origem = dados_final[ [ col for col in cols if 'ORIGEM' in col ] ]
dados_origem = dados_origem.rename( columns={ col:col.replace('AEROPORTO_DE_ORIGEM_','') for col in dados_origem.columns } )
dados_origem = dados_origem.rename( columns={ col:col.replace('_ORIGEM','') for col in dados_origem.columns } )

dados_destino = dados_final[ [ col for col in cols if 'DESTINO' in col ] ]
dados_destino = dados_destino.rename( columns={ col:col.replace('AEROPORTO_DE_DESTINO_','') for col in dados_destino.columns } )
dados_destino = dados_destino.rename( columns={ col:col.replace('_DESTINO','') for col in dados_destino.columns } )

In [29]:
dados_aeroportos = pd.concat( [ dados_origem,  dados_destino] )
dados_aeroportos = dados_aeroportos.drop_duplicates( 'SIGLA' )
dados_aeroportos = dados_aeroportos.set_index('SIGLA')
dados_aeroportos = dados_aeroportos.to_dict('index')

In [30]:
nx.set_node_attributes( graph_voos, dados_aeroportos )

**Visualizando dados**

In [31]:
graph_voos.nodes

NodeView(('SBAC', 'SBFZ', 'SBBE', 'SBBR', 'SBCF', 'SBCT', 'SBEG', 'SBGL', 'SBGO', 'SBGR', 'SBJE', 'SBJU', 'SBKP', 'SBMO', 'SBPA', 'SBPL', 'SBRF', 'SBRJ', 'SBSG', 'SBSL', 'SBSN', 'SBSP', 'SBSV', 'SBTC', 'SBTE', 'SBVT', 'SDSC', 'SDZG', 'SNIG', 'SNOB', 'SNWS', 'SWBE', 'SBAE', 'SBAQ', 'SBAR', 'SBAT', 'SBAU', 'SBBH', 'SBBV', 'SBCA', 'SBCB', 'SBCG', 'SBCH', 'SBCJ', 'SBCN', 'SBCR', 'SBCX', 'SBCY', 'SBDB', 'SBDN', 'SBDO', 'SBFI', 'SBFL', 'SBIL', 'SBIZ', 'SBJA', 'SBJD', 'SBJI', 'SBJP', 'SBJV', 'SBLO', 'SBMA', 'SBMG', 'SBMK', 'SBML', 'SBMQ', 'SBNF', 'SBPB', 'SBPF', 'SBPG', 'SBPJ', 'SBPS', 'SBPV', 'SBRB', 'SBRD', 'SBRP', 'SBSI', 'SBSJ', 'SBSR', 'SBTG', 'SBUL', 'SBVH', 'SBZM', 'SDCO', 'SSKW', 'SWLC', 'SBBU', 'SBCP', 'SBME', 'SDAG', 'SBKG', 'SBUR', 'SBJR', 'SDIM', 'SDTK', 'SDUB', 'SIMK', 'SJTC', 'SNBA', 'SBQV', 'SBSO', 'SNBR', 'SWGN', 'SBAX', 'SBGV', 'SBIP', 'SBVG', 'SNGA', 'SNPD', 'SNTF', 'SNVB', 'SBLJ', 'SBPO', 'SBTD', 'SSCT', 'SSFB', 'SSGY', 'SSKM', 'SSOG', 'SSPG', 'SSPI', 'SSUV', 'SSVL', 'SBBW'

In [32]:
graph_voos.is_directed()

True

## Salvando o grafo

**Salvando gml**

In [33]:
nx.write_gml(graph_voos, 'graph_br_flights.gml.gz')

**Testando arquivo**

In [34]:
g = nx.read_gml('graph_br_flights.gml.gz')

## Resumo final do grafo

Este notebook construiu um grafo Dirigido contendo informações de voos que ocorreram nos aeroportos brasileiros de 2000 a 2020.

Abaixo, uma lista das variáveis das arestas do grafo.
	
- **DISTANCIA_VOADA_KM**	: Distância entre origem e destino
- **VOOS** - Quantidade voos feitos entre os aeroportos
- **VOO_REGULAR**	: Quantidade de voos regulares feitas entre os aeroportos
- **VOO_NAO_REGULAR**	: Quantidade de voos não regulares feitas entre os aeroportos
- **IMPRODUTIVO**	: Quantidade de voos improdutivos feitas entre os aeroportos
- **EMPRESA_NACIONAL**	: Quantidade de voos de empresa nacional
- **NATUREZA_DOMESTICA**	: Quantidade de voos de natureza doméstica